# TP4 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

# Enunciado do Problema 1

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
 - A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  move-se (i.e. derrapa); no modo `Stopped` o veículo está imobilizado.
 - A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`   e a `velocidade linear das rodas`  ambas em relação so solo.
 - Assume-se que o sistema de travagem exerce uma força de atrito proporcional à diferença das duas velocidades.  A dinâmica contínua, as equações de fluxo, está descrita  abaixo.
 - Os “switchs” são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser  de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
 - É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

Pretende-se

 1. Defina um autómato híbrido que descreva a dinâmica do sistema segundo as notas abaixo indicadas e com os “switchs” por si escolhidos.
 2. Modele em lógica temporal linear LT  propriedades que caracterizam o comportamento desejável do sistema. Nomeadamente 
     1. ”o veículo imobiliza-se completamente em menos de $t$ segundos” 
     2. “a velocidade $V$ diminui sempre com o tempo”.
 3. Codifique em SMT’s o modelo que definiu em 1.
 4. Codifique em SMT’s a verificação das propriedades temporais que definiu em 2.

![](https://paper-attachments.dropboxusercontent.com/s_C5426E33053EA04D1E3446B871CC8485C43045FF7CAAA1FC86A9B3A8E09058F9_1574702123197_Documento.png)

Equações de Fluxo 

 1. Durante  a travagem não existe qualquer  força no sistema excepto as forças de atrito. Quando uma superfície se desloca em relação à outra, a força de atrito  é proporcional à força de compressão entre elas.
 2. No contacto rodas/solo o atrito é constante porque a força de compressão é o peso; tem-se $f = a\, P$ sendo  $a$ a constante de atrito e $P$ o peso. Ambos são fixos e independentes do modo.
 3. No contacto corpo/rodas,  a força de compressão é a força de travagem que aqui se assume como proporcional à diferença de velocidades

$$ F =  c\, (V-v) $$

A  constante de proporcionalidade $c$ depende do modo: é elevada no modo `Stopping` e baixa nos outros.

 4. As  equações que traduzem a dinâmica  do sistema são, em todos os modo excepto `Blocked`,

$$(\dot{V} \,=\, -F)\,\land\, (\dot{v} \,=\, -a\, P  + F)$$
                                                        
e , no modo `Blocked`,  a dinâmica do sistema é  regida por

$$(V = v) \;\land\; (\,\dot{v}\,=\, -a\, P\,)$$

 5. Tanto no modo `Blocked`  como no modo `Free`  existe um “timer” que impede que o controlo aí permaneça mais do que $\,\tau\,$segundos.  Os     $\mathsf{switch}(V,v,t,V',v',t')\,$ nesses modos devem forçar esta condição. 
 6. Todos os “switchs” devem ser construídos de  modo a impedir a existência de trajetórias de Zenão.
 7. No instante inicial assume-se $\,V = v\,=\,V_0$  ,  em que a velocidade $V_0$ é o “input” do problema.

# Análise

O objetivo de um sistema ABS é de evitar que as rodas de um carro bloqueiem quando uma travagem brusca ocorre. Neste caso, a travagem é tal que a única força exercida no sistema é a de atrito, seja entre o corpo e as rodas como as rodas e o solo.

# Implementação

Para a resolução do problema em questão, decidiu-se usar o módulo `pysmt.shortcuts`, com as funcionalidades possíveis para a utilização de um SMT Solver. Importam-se também os tipos deste Solver, a partir do módulo `pysmt.typing`.

In [17]:
from pysmt.shortcuts import *
import pysmt.typing
import itertools 

Geram-se os valores enumerados para cada um dos Modos de execução.

In [18]:
Mode, (START, FREE, STOPPING, BLOCKED, STOPPED) = EnumSort('Mode', ('START','FREE','STOPPING', 'BLOCKED','STOPPED'))

O "Timer" definido para o estado `BLOCKED` e `FREE` deve ser uma variável global do sistema que limita a sua operabilidade.

In [25]:
timer = 3

Declaram-se as variáveis para cada um dos estados.

In [19]:
def declare(i):
    s = {}
    s['T'] = Symbol('T'+str(i), Real)
    s['V'] = Symbol('V'+str(i), Real)
    s['R'] = Symbol('R'+str(i), Real)
    s['M'] = Symbol('M'+str(i), Mode)
    #s['c'] = Symbol('c'+str(i), Real)
    s['Timer'] = Symbol('Timer'+str(i), Real)
    return s

Define-se o predicado Inicial. Note-se a não necessidade de manipular a variável `Timer` quando esta não tem utilidade.

In [22]:
def init(s, vi):
    return And(Equals(s['T'], Real(0)), Equals(s['V'], vi), Equals(s['R'], vi), Equals(s['M'], START))

In [23]:
def untimed_trans(c, p):
    startToStopping = And(Equals(s['M'],START), Equals(p['M'],STOPPING), Equals(s['T'],p['T']), Equals(s['V'],p['V']), Equals(s['R'],p['R']))
    
    stoppingToBlocked = And(Equals(s['M'],STOPPING), Equals(p['M'],BLOCKED), Equals(s['T'],p['T']), s['V']>0, s['R']>=0, 
                            Equals(s['V'],p['V']), Equals(s['R'],p['R']), Equals(p['Timer'],0), s['V']-s['R']<e)
    
    blockedToFree = And(s['M']==BLOCKED, p['M']==FREE ,s['T']==p['T'],s['V']>0,s['R']>=0, 
                       s['V']==p['V'], s['R']==p['R'], s['Timer']>=tau, 
                       p['Timer']==0)
    
    
    freeToStopping  =  And(s['M']==FREE, p['M']==STOPPING ,s['T']==p['T'], s['V']>0,s['R']>=0, 
                          s['V']==p['V'], s['R']==p['R'], s['Timer']>=tau)
    
    
    stoppingToStopped = And(s['M']==STOPPING, p['M']==STOPPED ,s['T']==p['T'],
                           s['V']<e, s['R']<e, p['V'] == 0, p['R'] == 0)
    
    freeToStopped = And(s['M']==FREE, p['M']==STOPPED ,s['T']==p['T'],
                           s['V']<e, s['R']<e, p['V'] == 0, p['R'] == 0)
    
    blockedToStopped = And(s['M']==BLOCKED, p['M']==STOPPED ,s['T']==p['T'],
                           s['V']<e, s['R']<e, p['V'] == 0, p['R'] == 0)
    
    stoppedToStopped = And(s['M']==STOPPED, p['M']==STOPPED, s['T'] == p['T'], 
                          s['V']==p['V'], s['R']== p['R'])
    
    
    Or(startToStopping, stoppingToBlocked)